# 第五节 实战召回优化——节点组

上一节我们讲了大模型查询重写这个优化技巧，这一节内容我们将在代码层面实现优化召回率的其中一种方式——节点组。

## 提升指标的优化策略之节点组

**节点组**这个事儿，我是真的踩过坑。之前有个项目，**文档切得特别碎，一段话能切成三四块，结果召回的时候经常是前言不搭后语**。后来重新调整了分块策略——**把相关的内容尽量放在一个 chunk 里**，效果立竿见影。我现在的经验是，与其追求固定长度的分块，不如按主题来切，哪怕有的块大点有的块小点。

那么如何从**代码层面实现节点组优化策略**呢？

## 节点组优化

你还记得 Retriever 是怎么定义的吗？

```python
retriever = Retriever(
    doc=documents, group_name="CoarseChunk", similarity="bm25_chinese", topk=3
)  # 定义检索组件
```

LazyLLM 的 Retriever 设计挺巧妙的，它把检索拆成了三个关键部分。Node Group（对应的就是`group_name`）**负责文档怎么切**，Similarity**管相关性怎么算**，Index 管**检索快不快**。这三个东西就像是做菜的食材、调料和火候，缺一不可。

![image](../assets/image-20250907165046-8s8qtzf.png)

说到 Node Group，这玩意儿其实就是**决定你检索的最小单位是什么**。我之前有个项目，客户的文档特别长，一篇技术手册能有上百页。如果**不切分，检索出来就是整篇文档**，根本没法用。但**切太碎了也不行**——我试过按句子切，结果经常出现"因此..."、"如上所述..."这种没头没尾的句子。

LazyLLM 的做法挺聪明，它先把文档读进来形成 Origin Node Group，就是**原始的大块文本**。然后你可以**根据需要创建不同粒度的节点组**。系统会先完整读进来，然后你可以按句号切分，每个句子变成一个独立的节点，但它会记住父节点是谁。这样**即使检索到某个句子，也能追溯回原文**。

最让我觉得实用的是它的**懒加载机制**。你定义了转换规则后，节点不会立即生成，而是等到真正需要的时候才创建。这在处理大规模文档库的时候特别有用。

**相似度计算**这块，BM25 和向量检索各有千秋。**BM25 就像是个老实人，关键词匹配特别准**，用户搜"错误代码 404"，它肯定能找到包含这些词的文档。但**向量检索更懂语义**，用户问"网页打不开"，它能理解这可能跟 404 错误相关。我现在的做法是两个都用，然后根据场景调权重。

LazyLLM 还提供了**几个预设的分块方案**。**CoarseChunk 是 1024 字符一块，重合长度为 100，适合需要较完整上下文的场景**；**MediumChunk 是 256 字符一块，重合长度为 25；FineChunk 只有 128 字符，重合长度为 12**，适合精确匹配。我一般会创建多个不同粒度的节点组，检索的时候多路召回，这样既能保证召回率，又不会丢失上下文。

还有个 LLMParser 的功能我觉得挺有意思，它可以**用大模型对文本进行转换，比如提取摘要或者生成问答对**。我在一个 FAQ 系统里用过这个，先让模型把文档转成问答对形式，检索效果提升特别明显。不过这个比较耗资源，大规模使用要考虑成本。

有个细节特别重要——节点之间的关联关系。LazyLLM 会记录父子节点关系，这样你检索到一个细粒度的节点后，可以往上找到它的完整段落，也可以往下找到所有相关的子节点。这个功能在做多跳问答的时候特别有用。

### 多个节点组协同

那么问题来了，我们能不能**实现多个节点组**呢？请看代码：

In [ ]:
pip install lazyllm
export QWEN_API_KEY='sk-这里填写你的key'

In [ ]:
from lazyllm import Document, Retriever, SentenceSplitter

docs = Document("./data_kb")

# 使用内置 SentenceSplitter 创建新的 node group
# 此处 chunk_size 和 chunk_overlap 会透传给 SentenceSplitter
# 最终输出的分块规则为 长度为 512，重叠为 64
docs.create_node_group(name='512Chunk', transform=SentenceSplitter, chunk_size=512, chunk_overlap=64)

# 查看节点组内容，此处我们通过一个检索器召回一个节点并打印其中的内容，后续都通过这个方式实现
group_names = ["CoarseChunk", "MediumChunk", "FineChunk", "512Chunk"]
for group_name in group_names:
    retriever = Retriever(docs, group_name=group_name, similarity="bm25_chinese", topk=1)
    node = retriever("亚硫酸盐有什么作用？")
    print(f"======= {group_name} =====")
    print(node[0].get_content())

上面的`group_names`实际上使用了三个系统的节点组，`CoarseChunk`、`MediumChunk`和`FineChunk`，还有一个自定义的节点组`512Chunk`。节点组支持以下参数：

- ​`name` （str, default: None）: 新的节点组的名称
- ​`transform`（Callable）： 节点组**解析规则**，函数原型是 `(DocNode, group_name, **kwargs) -> List[DocNode]`。LazyLLM 内置了 `SentenceSplitter`，用户也传入可调用对象实现**自定义转换规则**。
- ​`parent`（str, default: LAZY_ROOT_NAME）：**基于哪个节点组进行解析**，默认情况下基于全文节点组（即上文提到的 origin）进行构造，用户可以指定该参数实现更高效的节点组构造
- ​`trans_node`（bool, default: None ) 决定了 transform 的输入和输出是 DocNode 还是 str ，默认为 None。只有在 transform 为 Callable 时才可以设置为 true。
- ​`num_workers`（int，default：0）：Transform 时所用的新线程数量，默认为 0
- ​`kwargs`：和具体实现相关的参数，会透传至 transform 函数。

### 自定义节点组分块策略

聪明的你可能明白了，**实际上节点组真正发挥功能的核心是**​**​`transform`​**​**函数**。既然这样，是不是可以自定义`transform`​**函数呢**？

当然是可以的，下面的代码实现了**基于句号进行分块的节点组**：

In [ ]:
from lazyllm import Document, Retriever
from lazyllm.tools.rag.doc_node import DocNode

docs = Document("./data_kb")


# 第一种：函数实现直接对字符串进行分块规则
def split_by_sentence1(node: str, **kwargs):
    """函数接收字符串，返回字符串列表，输入为trans_node=False时调用"""
    return node.split("。")


docs.create_node_group(name="block1", transform=split_by_sentence1)


# 第二种：函数实现获取DocNode对应文本内容后进行分块，并构造DocNode
# 适用于返回非朴素DocNode，例如LazyLLM提供了ImageDocNode等特殊DocNode
def split_by_sentence2(node: DocNode, **kwargs):
    """函数接收DocNode，返回DocNode列表，输入为trans_node=False时调用"""
    content = node.get_text()
    nodes = []
    for text in content.split("。"):
        nodes.append(DocNode(text=text))
    return nodes


docs.create_node_group(name="block2", transform=split_by_sentence2, trans_node=True)


# 第三种：实现了 __call__ 函数的类
# 优点是一个类用于多种分块，例如这个例子可以通过控制实例化时的参数实现基于多种符号的分块
class SymbolSplitter:
    """实例化后传入Transform，默认情况下接收字符串，trans_node为true时接收DocNode"""

    def __init__(self, splitter="。", trans_node=False):
        self._splitter = splitter
        self._trans_node = trans_node

    def __call__(self, node):
        if self._trans_node:
            return node.get_text().split(self._splitter)
        return node.split(self._splitter)


sentence_splitter_1 = SymbolSplitter()
docs.create_node_group(name="block3", transform=sentence_splitter_1)

# 指定传入 DocNode
sentence_splitter_2 = SymbolSplitter(trans_node=True)
docs.create_node_group(name="block4", transform=sentence_splitter_2, trans_node=True)

# 指定分割符号为 \n
paragraph_splitter = SymbolSplitter(splitter="\n")
docs.create_node_group(name="block5", transform=paragraph_splitter)

# 第四种：直接传入lambda函数，适用于简单规则情况
docs.create_node_group(name="block6", transform=lambda b: b.split("。"))

# 查看节点组内容，此处我们通过一个检索器召回一个节点并打印其中的内容，后续都通过这个方式实现
for i in range(6):
    group_name = f"block{i + 1}"
    retriever = Retriever(
        docs, group_name=group_name, similarity="bm25_chinese", topk=1
    )
    node = retriever("亚硫酸盐有什么作用？")
    print(f"======= {group_name} =====")
    print(node[0].get_content())

### 小块召回大块

可能你会问了，能不能**实现通过小块召回大块**，也就是召回到某个节点时，取出该节点的内容，同时取出该节点的父节点？当然可以，下面是示例代码：

In [ ]:
import lazyllm
import os
from lazyllm import Document, Retriever, OnlineChatModule

# 创建在线大模型对象，指定来源、模型和API密钥
llm = OnlineChatModule(
    source="qwen",
    model="qwen-plus-latest",
    api_key=os.getenv("QWEN_API_KEY"),
)  # 调用大模型

prompt = "你是一个友好的 AI 问答助手，你需要根据给定的上下文和问题提供答案。\
          根据以下资料回答问题：\
          {context_str} \n "

robot = llm.prompt(lazyllm.ChatPrompter(instruction=prompt, extra_keys=["context_str"]))

# 文档加载
docs = Document("./data_kb")
docs.create_node_group(
    name="sentences",
    transform=(lambda d: d.split("\n") if d else ""),
    parent=Document.CoarseChunk,
)

# 定义两个不同的检索器在同一个节点组使用不同相似度方法进行检索
retriever = Retriever(docs, group_name="sentences", similarity="bm25_chinese", topk=3)

# 执行查询
query = "都有谁参加了2008年的奥运会？"

# 原节点检索结果
doc_node_list = retriever(query=query)
doc_node_res = "".join([node.get_content() for node in doc_node_list])
print(f"原节点检索结果：\n{doc_node_res}")
print("=" * 100)

# 父节点对应结果
parent_list = [node.parent.get_text() for node in doc_node_list]
print(f"父节点检索结果：\n{''.join(parent_list)}")
print("=" * 100)

# 将query和召回节点中的内容组成dict，作为大模型的输入
res = robot(
    {"query": query, "context_str": "".join([node_text for node_text in parent_list])}
)

print("系统答案：\n", res)

### 基于 LLM 的节点组

除了上述固定长度分块方法，LazyLLM 提供了 LLMParser 可以实现基于 LLM 对文档进行解析的操作，支持三种模式：

- **摘要提取**（summary）：对文段内容进行分析，提炼出核心信息，生成简洁且能代表全文主旨的摘要，帮助用户快速获取关键信息。
- **关键词提取**（keyword）：自动从文段中识别出最具代表性的关键词，以便后续检索、分类或分析。
- **QA 对提取**：从文段中自动抽取多个问答对，匹配与用户查询相似的问题，从而提供预设答案。这些问答对可以用于问答系统、知识库建设或生成式 AI 组件的参考，以提高信息获取的效率和准确性。

In [ ]:
from lazyllm import OnlineChatModule, LLMParser, Document, Retriever
import os

# 创建在线大模型对象，指定来源、模型和API密钥
llm = OnlineChatModule(
    source="qwen",
    model="qwen-plus-latest",
    api_key=os.getenv("QWEN_API_KEY"),
    stream=False
)  # 调用大模型

# LLMParser 是 LazyLLM 内置的基于 LLM 进行节点组构造的类，支持 summary，keywords和qa三种
summary_llm = LLMParser(llm, language="zh", task_type="summary")  # 摘要提取LLM
keyword_llm = LLMParser(llm, language="zh", task_type="keywords")  # 关键词提取LLM
qapair_llm = LLMParser(llm, language="zh", task_type="qa")  # 问答对提取LLM


# 利用 LLMParser 创建节点组
docs = Document("./data_kb")

docs.create_node_group(
    name="summary", transform=lambda d: summary_llm(d), trans_node=True
)
docs.create_node_group(
    name="keyword", transform=lambda d: keyword_llm(d), trans_node=True
)
docs.create_node_group(
    name="qapair", transform=lambda d: qapair_llm(d), trans_node=True
)

# 查看节点组内容，此处我们通过一个检索器召回一个节点并打印其中的内容，后续都通过这个方式实现
group_names = ["CoarseChunk", "summary", "keyword", "qapair"]
for group_name in group_names:
    retriever = Retriever(
        docs, group_name=group_name, similarity="bm25_chinese", topk=1
    )
    node = retriever("亚硫酸盐有什么作用？")
    print(f"======= {group_name} =====")
    print(node[0].get_content())

### 综合使用多个节点组

在实际应用中通常是**选择上述几种节点组中的多个节点组进行混合检索**，召回多组信息后进行拼接等后处理获取更全面、准确的上下文。

在 RAG 系统中使用节点组并召回对应节点组的方式如下方代码所示，修改对应创建方法和检索节点组名称即可实现在不同节点组进行召回的目的。

In [ ]:
import lazyllm
import os

# 创建在线大模型对象，指定来源、模型和API密钥
llm = OnlineChatModule(
    source="qwen",
    model="qwen-plus-latest",
    api_key=os.getenv("QWEN_API_KEY"),
)

# 文档加载
documents = lazyllm.Document(dataset_path="./data_kb")
documents.create_node_group(name='sentences', transform=lambda b: b.split('。'))
documents.create_node_group(name='block', transform=lambda b: b.split('\n'))

# 检索组件定义
retriever1 = lazyllm.Retriever(doc=documents, group_name="block", similarity="bm25_chinese", topk=3)
retriever2 = lazyllm.Retriever(doc=documents, group_name="sentences", similarity="bm25_chinese", topk=3)

# prompt 设计
prompt = '你是一个友好的 AI 问答助手，你需要根据给定的上下文和问题提供答案。\
          根据以下资料回答问题：\
          {context_str} \n '
robot = llm.prompt(lazyllm.ChatPrompter(instruction=prompt, extra_keys=['context_str']))

# 推理
query = "亚硫酸盐有什么作用？"
# 将Retriever组件召回的节点全部存储到列表doc_node_list中
doc_node_list1 = retriever1(query=query)
doc_node_list2 = retriever2(query=query)

# 将两个检索器的召回结果合并到一个列表
doc_node_list = doc_node_list1 + doc_node_list2

# 将query和召回节点中的内容组成dict，作为大模型的输入
res = robot({"query": query, "context_str": "".join([node.get_content() for node in doc_node_list])})

print("系统答案：", res)

总之，由于文档库的类型和内容差异，**不同的文档库、不同类型的文档内容适用的节点组会有所不同**。例如对于本课程构造的 CMRC 文档库而言，每个段落的内容相互独立，因此段落分块有助于完整召回问题的上下文，对于法律法规等每个句子**包含大量信息量，且每个句子描述的内容可能相互独立的文档**来说，句子节点组是更优的选择。因此具体任务中使用何种节点组进行检索召回需要通过观察文档库特点并结合实际召回表现进行选择。